In [24]:
import time, os
import numpy as np
from IPython.display import Audio

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.callbacks import History

In [25]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4722188080881839248
]


In [26]:
import soundfile as sf
channels = 2
dataX, samplerate = sf.read('../CustomSongs/7388-7168/Chop Suey/song.ogg')
sf.write('../CustomSongs/7388-7168/Chop Suey/song.wav', dataX, samplerate)

In [27]:
#Sandbox for onset generation
import scipy.io.wavfile
import onsetdetection
sr, audio = scipy.io.wavfile.read('../CustomSongs/7388-7168/Chop Suey/song.wav')
#audio = audio[:,0] # make it mono

In [28]:
import json
infile = open('../CustomSongs/7388-7168/Chop Suey/Hard.json', 'r')
dataY = json.loads(infile.read())
infile.close()

In [29]:
trainY = np.array([[dataY['_notes'][i]['_lineIndex'], dataY['_notes'][i]['_lineLayer'], dataY['_notes'][i]['_type'], dataY['_notes'][i]['_cutDirection']] for i in range(len(dataY['_notes']))])

In [30]:
dataY['_notes'][0]

{'_time': 16.058334350585938,
 '_lineIndex': 3,
 '_lineLayer': 0,
 '_type': 1,
 '_cutDirection': 7}

In [31]:
songTicks = dataX.shape[0]/samplerate
songNoteTicks = dataY['_notes'][-1]['_time']

tickRate = int((songTicks/songNoteTicks) * 44100)

def getDataRunway(t = 0):
    return dataX[int((t-1)*tickRate):int((t)*tickRate)]

In [32]:
set([dataY['_notes'][i]['_type'] for i in range(len(dataY['_notes']))])

{0, 1}

In [33]:
#generate training output
dataRows, dataCols = 3, 4

tY = {}
tX = {}
for p in dataY['_notes']:
    if p['_type'] is 3:
        continue
    t = p['_time']
    
    d = p['_cutDirection']
    d -= (1 * (d%2))
    d /= 2
    dX, dY = 0, 0
    if d is 0:
        dX, dY = 0, 1
    elif d is 1:
        dX, dY = 1, 0
    elif d is 2:
        dX, dY = 0, -1
    else:
        dX, dY = -1, 0
    
    if t in tY:
        _img = tY[t]
        _img[p['_lineLayer']][p['_lineIndex']][1] = dY
        _img[p['_lineLayer']][p['_lineIndex']][2] = dX
        _img[p['_lineLayer']][p['_lineIndex']][0] = (2 * p['_type']) - 1
        tY.update({t: _img})
    else:
        _img = np.full((3,4,3), 0)
        
        #CUT DIRECTION (Y, X)
        _img[p['_lineLayer']][p['_lineIndex']][1] = dY
        _img[p['_lineLayer']][p['_lineIndex']][2] = dX
    
        #COLOR
        _img[p['_lineLayer']][p['_lineIndex']][0] = (2 * p['_type']) - 1
        
        tY.update({t: _img})
        tX.update({t: getDataRunway(t)})
   
#print(tY[4].shape)
#print(tX[4].shape)

In [34]:
len(tY)

512

In [37]:
X, Y = [], []
for k in tY.keys():
    X += [tX[k]]
    Y += [tY[k]]
X, Y = np.array(X), np.array(Y)
#Y = Y.reshape(Y.shape[0], Y.shape[-2], Y.shape[-1], -1)
Y.shape

(512, 3, 4, 3)

In [38]:
o=Sequential()
o.add(
    keras.layers.SimpleRNN(Y.shape[-1] * Y.shape[-2], 
                           activation='tanh', 
                           dropout=0.0, 
                           recurrent_dropout=0.0,
                           stateful=False, 
                           unroll=False,
                          input_shape=(X.shape[1], channels))
)
o.add(
    keras.layers.Reshape((1, 3, 4), input_shape = (Y.shape[-1] * Y.shape[-2], ))
)
o.add(
    keras.layers.Conv2DTranspose(filters=3, 
                                 kernel_size=(3,2), 
                                 strides=(1, 1), 
                                 padding='valid', 
                                 data_format=None, 
                                 dilation_rate=(1, 1), 
                                 activation='tanh')
)


In [ ]:
sgd = keras.optimizers.SGD(lr=0.01, nesterov=True)
o.compile(optimizer=sgd, loss='mse')
hist = o.fit(X, Y,
          batch_size=2,
          epochs=3)#,
          #validation_split=0.2)
#~1.5 - 1.6

Epoch 1/3
 16/512 [..............................] - ETA: 7:33 - loss: 0.0857

In [ ]:
sendMessage('7039154386@vtext.com', 'Finished training\nLoss: {}'.format(hist.history['loss'][-1]))

In [83]:
testout = o.predict(X[:])

In [90]:
(testout[0])# * 5).astype('int')

array([[[-0.11740984, -0.01755702, -0.22677885],
        [ 0.05784214,  0.10615101, -0.11361618],
        [ 0.05620984,  0.08970454,  0.15112035],
        [-0.13384372, -0.10968328, -0.09411307]],

       [[ 0.0109544 , -0.12744291, -0.00914534],
        [ 0.08595253, -0.09595299, -0.09834012],
        [ 0.20944545,  0.01809225, -0.21826468],
        [ 0.20607492,  0.13200557, -0.16858278]],

       [[ 0.01375662,  0.1589484 , -0.04385418],
        [ 0.04692488, -0.07804783,  0.03734485],
        [-0.2901995 , -0.19850571,  0.16995184],
        [-0.04504027,  0.05152607,  0.04650337]]], dtype=float32)

In [98]:
dataY['_notes'][0]

{'_time': 4, '_lineIndex': 1, '_lineLayer': 0, '_type': 0, '_cutDirection': 1}

In [99]:
newnotes = []
color_threshold = np.average(np.abs(testout[:][:,:,0]))
for i in range(testout.shape[0]):
    for row in range(testout.shape[1]):
        for col in range(testout.shape[2]):
            p = testout[i][row, col]
            if abs(p[0]) > color_threshold:
                dat = {}
                dat['_time'] = i
                dat['_lineIndex'] = row
                dat['_lineLayer'] = col
                    
                dat['_type'] = 0 if p[0] < 0 else 1
                
                if abs(p[1]) > abs(p[2]):
                    dat['_cutDirection'] = 0 if p[1] > 0 else 4
                else:
                    dat['_cutDirection'] = 2 if p[2] > 0 else 6
                
                newnotes += [dat]
                

In [103]:
outY = dataY.copy()
outY['_notes'] = newnotes

In [106]:
outfile = open('gen.json', 'w')
outfile.write(json.dumps(outY))
outfile.close()

In [78]:
import smtplib
import base64
import time
from time import sleep

GMAIL_USR = "m1raszp1e@gmail.com"
GMAIL_PSW = "r@spberry"

msgHeader = '[@vt]'

def sendMessage(to, msg):
    smtps = smtplib.SMTP("smtp.gmail.com", 587)
    smtps.ehlo()
    smtps.starttls()
    smtps.ehlo
    smtps.login(GMAIL_USR, GMAIL_PSW)
    h = 'To:' + to + '\n' + 'From: ' + GMAIL_USR + '\n' + 'Subject: ' + '\n'
    m = h + '\n ' + msgHeader + ' ' + msg + '\n'
    smtps.sendmail(GMAIL_USR, to, m)
    smtps.close()
    print("Message sent to " + to)

mail = None
def mailLogin():
    m = imaplib.IMAP4_SSL('imap.gmail.com')
    m.login(GMAIL_USR, GMAIL_PSW)
    result = m.select("INBOX")
    print("MAILBOX_SELECT: " + result[0])
    return m
